In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']=str(1)

import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import csv
from keras.models import Sequential, Model
from keras.layers import Conv2D, ConvLSTM2D, Dense, MaxPooling2D, Dropout, Flatten, Reshape, merge, Input
from keras.optimizers import Adam

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('image_dir', '/notebooks/udacity/car_data/data/IMG/', 'Simulator Image data')
flags.DEFINE_string('data_path', '/notebooks/udacity/car_data/data/driving_log.csv', 'Simulator CSV')
flags.DEFINE_float('learn_rate', 0.0001, 'Trainign learning rate')

print ('Init completed')

with open(FLAGS.data_path, 'r') as f:
    reader = csv.reader(f)
    csv = np.array([row for row in reader][1:])
# center image, left image, right image
# steering (-0.8, 0.8)
# throttle (0, 1)
# brake (0, 1)
# speed (0, 9.8)

# Process single image
def proc_img(img): # input is 160x320x3
    img = img[59:138:2, 0:-1:2, :] # select vertical region and take each second pixel to reduce image dimensions
    img = (img / 127.5) - 1.0 # normalize colors from 0-255 to -1.0 to 1.0
    return img # return 40x160x3 image

# Read image names and remove IMG/ prefix

from get_images import get_images

datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

images=get_images(datadirs,0.08)
images=images[images.inverse==1]
images=images[images.side!=0]
image_names_full, y_data_full = images.img.values, images.real.values

# Random sort for data and split test and validation sets
def newRandomTestValidationSplit(X, y, inverse):
    X_tr, X_val, y_tr, y_val, inverse_tr, inverse_test = train_test_split(X, y,inverse, test_size=0.01, random_state=111)
    return X_tr, X_val, y_tr, y_val,inverse_tr, inverse_test

# Batch generator for training data
def generate_image_batch_tr(names, y_data, batch_size = 32):
    total_items = len(names)
    curr_item = 0
    while (True):
        image_data = np.zeros((batch_size,40,160, 3),dtype=float)
        steering_data = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            image_name = names[curr_item]
            image = mpimg.imread(image_name)
            image_data[j] = proc_img(image)
            steering_data[j] = y_data[curr_item]
            curr_item = (curr_item+1)%total_items
        yield image_data, steering_data
        


# Batch generator for validation data (in this implementation same as for training data)
generate_image_batch=generate_image_batch_tr


Using TensorFlow backend.


Init completed


/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(40,160,3))
# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution layer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x5 = Dropout(0.5)(x5)
x6 = Dense(128, activation='relu')(x5)
x6 = Dropout(0.5)(x6)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 40, 160, 3)    0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 40, 160, 3)    12          input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 20, 80, 3)     0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 20, 80, 32)    896         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [3]:
from sklearn.utils import shuffle
import cv2

def generator(names, y, inverse, batch_size = 32, preprocessing = lambda x:x):
    total_items = len(names)
    while (True):
        names, y, inverse =shuffle(names, y, inverse)
        
        for i in range(int(len(names)/batch_size-1)):
            images=[]
            steering=[]
            for j in range(batch_size):
                now=i*batch_size+j
                image=cv2.imread(names[now])
                if inverse[now]==-1:
                    image=np.fliplr(image)
                images.append(preprocessing(image))
                steering.append(y[now]*inverse[now])
            yield np.array(images), np.array(steering)

In [ ]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images_generator2.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
X_tr_names, X_val_names, y_tr, y_val,inverse_tr, inverse_test = \
    newRandomTestValidationSplit(image_names_full, y_data_full, [1 for i in image_names_full])

print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generator(X_tr_names, y_tr,inverse_tr,  64,preprocessing=proc_img),\
                              samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generator(X_val_names, y_val,inverse_test, 32,preprocessing=proc_img),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
90944/90953 [============================>.] - ETA: 0s - loss: 0.0788 - mean_squared_error: 0.0788   

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.05770, saving model to model_test_images_generator2.h5
91008/90953 [==============================] - 80s - loss: 0.0787 - mean_squared_error: 0.0787 - val_loss: 0.0577 - val_mean_squared_error: 0.0577
Epoch 2/100
91008/90953 [==============================] - 77s - loss: 0.0646 - mean_squared_error: 0.0646 - val_loss: 0.0591 - val_mean_squared_error: 0.0591
Epoch 3/100
91008/90953 [==============================] - 77s - loss: 0.0593 - mean_squared_error: 0.0593 - val_loss: 0.0530 - val_mean_squared_error: 0.0530
Epoch 4/100
91008/90953 [==============================] - 77s - loss: 0.0561 - mean_squared_error: 0.0561 - val_loss: 0.0516 - val_mean_squared_error: 0.0516
Epoch 5/100
91008/90953 [==============================] - 77s - loss: 0.0533 - mean_squared_error: 0.0533 - val_loss: 0.0537 - val_mean_squared_error: 0.0537
Epoch 6/100
91008/90953 [==============================] - 77s - loss: 0.0512 - mean_squared_error: 0.0

In [ ]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
#X_tr_names, X_val_names, y_tr, y_val = newRandomTestValidationSplit(image_names_full, y_data_full)
X_tr_names, X_val_names, y_tr, y_val,inverse_tr, inverse_test = \
    newRandomTestValidationSplit(image_names_full, y_data_full, [1 for i in image_names_full])
    
print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generate_image_batch_tr(X_tr_names, y_tr, 64),samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generate_image_batch(X_val_names, y_val, 32),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
90944/90953 [============================>.] - ETA: 0s - loss: 0.0259 - mean_squared_error: 0.0259  

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.04461, saving model to model_test_images.h5
91008/90953 [==============================] - 96s - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.0446 - val_mean_squared_error: 0.0446
Epoch 2/100
91008/90953 [==============================] - 96s - loss: 0.0216 - mean_squared_error: 0.0216 - val_loss: 0.0429 - val_mean_squared_error: 0.0429
Epoch 3/100
91008/90953 [==============================] - 96s - loss: 0.0196 - mean_squared_error: 0.0196 - val_loss: 0.0429 - val_mean_squared_error: 0.0429
Epoch 4/100
91008/90953 [==============================] - 95s - loss: 0.0179 - mean_squared_error: 0.0179 - val_loss: 0.0429 - val_mean_squared_error: 0.0429
Epoch 5/100
91008/90953 [==============================] - 96s - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0426 - val_mean_squared_error: 0.0426
Epoch 6/100
91008/90953 [==============================] - 95s - loss: 0.0153 - mean_squared_error: 0.0153 - val_l

In [14]:
next(q)

(array([[[[  0,  12,   9],
          [ 10,  29,  26],
          [ 30,  49,  46],
          ..., 
          [ 11,  16,  15],
          [ 12,  17,  16],
          [ 13,  18,  17]],
 
         [[  5,  26,  23],
          [  8,  29,  26],
          [ 20,  39,  36],
          ..., 
          [ 12,  17,  16],
          [ 13,  18,  17],
          [ 14,  19,  18]],
 
         [[ 16,  37,  34],
          [  8,  29,  26],
          [  5,  24,  21],
          ..., 
          [ 14,  19,  18],
          [ 15,  20,  19],
          [ 15,  20,  19]],
 
         ..., 
         [[117, 123, 128],
          [118, 124, 129],
          [119, 125, 130],
          ..., 
          [197, 194, 190],
          [186, 183, 179],
          [186, 183, 179]],
 
         [[122, 128, 133],
          [125, 131, 136],
          [126, 132, 137],
          ..., 
          [189, 184, 181],
          [196, 191, 188],
          [197, 192, 189]],
 
         [[120, 126, 131],
          [124, 130, 135],
          [125, 131, 136],

In [24]:
history.history

{'loss': [0.078073925246994122,
  0.064945485179591533,
  0.059714200041936671,
  0.057032494330922806,
  0.05410989002834183,
  0.052086850984543354,
  0.049928564816692375,
  0.047861373367681363,
  0.045719351660088225,
  0.043878138730755668,
  0.04199534048537204,
  0.040322387378804295,
  0.038822220612196812,
  0.037130849996552855,
  0.035782216053256967,
  0.03439293977570014],
 'mean_squared_error': [0.078073925246994122,
  0.064945485179591533,
  0.059714200041936671,
  0.057032494330922806,
  0.05410989002834183,
  0.052086850984543354,
  0.049928564816692375,
  0.047861373367681363,
  0.045719351660088225,
  0.043878138730755668,
  0.04199534048537204,
  0.040322387378804295,
  0.038822220612196812,
  0.037130849996552855,
  0.035782216053256967,
  0.03439293977570014],
 'val_loss': [0.060535559302260136,
  0.059001169392261014,
  0.054973017745490731,
  0.055459015970600059,
  0.048476462156094351,
  0.052164559220445564,
  0.045519070764040125,
  0.050753627036666048,
  